# 1. Install necessary libraries

!conda install -c conda-forge ffmpeg

!python.exe -m pip install -U demucs SoundFile

!conda install -c conda-forge ipython

# 2. Import required modules

In [11]:
import os
import sys
import subprocess
from pathlib import Path
from IPython.display import Audio, display

# 3. Define functions

In [15]:
def separate_with_demucs(file_path: str, output_dir: Path):
    """Performs separation of the file using Demucs and saves the result in output_dir"""
    audio_path = Path(file_path)
    if not audio_path.exists():
        print(f"❌ File does not exist: {audio_path}")
        return

    output_dir.mkdir(parents=True, exist_ok=True)

    print(f"🎧 Running Demucs separation for file: {audio_path.name}\n")

    command = [
        sys.executable, "-m", "demucs",
        "-n", "htdemucs",
        str(audio_path)
    ]

    try:
        subprocess.run(command, check=True)

    except subprocess.CalledProcessError as e:
        print("❌ Error while running Demucs:", e)
        return

    # Looking for results in output_dir/htdemucs/<file_name> (default Demucs location)
    demucs_base = Path(output_dir) / "htdemucs" / audio_path.stem
    if not demucs_base.exists():
        print("❌ Demucs results not found:", demucs_base)
        return

    # Move all .wav files to our folder ./separated/<song_name>/
    for wav in demucs_base.glob("*.wav"):
        dest = output_dir / wav.name
        wav.replace(dest)
        print(f"✅ Saved: {dest}")

    # Remove the original Demucs folder (optional)
    try:
        for f in demucs_base.iterdir():
            f.unlink()
        demucs_base.rmdir()
    except Exception:
        pass

    print(f"\n🎵 Finished! All tracks saved in: {output_dir}\n")

# 4. Run separation on a sample file

In [ ]:
# file_to_separate = "../musdb18/test/Al James - Schoolboy Facination.stem.mp4"
file_to_separate = "../musdb18/test/Al James - Schoolboy Facination.stem.mp4"
output_dir = Path.cwd().parent / "separated" / Path(file_to_separate).stem

if Path(file_to_separate).exists():
    separate_with_demucs(file_to_separate, output_dir)
else:
    print(f"❌ Sample file does not exist: {file_to_separate}. Please provide a valid audio file path.")

# 5. Play separated tracks

In [ ]:
if 'output_directory' in locals() and output_dir is not None and output_dir.exists():
    for stem_file in sorted(output_dir.glob("*.wav")):
        print(f"--- {stem_file.name} ---")
        display(Audio(str(stem_file)))